<a href="https://colab.research.google.com/github/QuantLet/ATSSB-Applied-Time-Series-Solutions-Book/blob/main/ATSSB_ARCH_Effects/ATSSB_ARCH_effects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install PythonTsa
! pip install arch
! pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 14.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.api import qqplot
from arch import arch_model
from scipy.stats import norm, jarque_bera, skew, kurtosis
from PythonTsa.LjungBoxtest import plot_LB_pvalue
from PythonTsa.Selecting_arma2 import choose_arma2
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ------------------------ Utility Functions ------------------------

def save_figure(fig, filename):
    """Save figure with transparent background."""
    fig.patch.set_alpha(0.0)
    for ax in fig.axes:
        ax.patch.set_alpha(0.0)
    fig.savefig(filename, dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

def style_legend_outside_bottom(ax, ncol=3):
    """Style legend: transparent, outside bottom."""
    legend = ax.legend(
        loc='upper center',
        bbox_to_anchor=(0.5, -0.15),
        ncol=ncol,
        frameon=True,
        fancybox=False,
        edgecolor='gray'
    )
    legend.get_frame().set_alpha(0.0)
    return legend

def acf_pacf_fig_save(series, lags=48, both=True, filename="acf_pacf.png"):
    """Plot ACF/PACF with transparent background and legend outside bottom."""
    if both:
        fig, ax = plt.subplots(2, 1, figsize=(10, 8))
        plot_acf(series, lags=lags, ax=ax[0], title="ACF")
        plot_pacf(series, lags=lags, ax=ax[1], title="PACF")
        # Add legends outside bottom for each subplot
        ax[0].legend(['ACF', '95% CI'], loc='upper center',
                     bbox_to_anchor=(0.5, -0.15), ncol=2, framealpha=0.0)
        ax[1].legend(['PACF', '95% CI'], loc='upper center',
                     bbox_to_anchor=(0.5, -0.15), ncol=2, framealpha=0.0)
    else:
        fig, ax = plt.subplots(figsize=(10, 4))
        plot_acf(series, lags=lags, ax=ax, title="ACF")
        ax.legend(['ACF', '95% CI'], loc='upper center',
                  bbox_to_anchor=(0.5, -0.15), ncol=2, framealpha=0.0)

    fig.tight_layout(rect=[0, 0.1, 1, 0.95])
    save_figure(fig, filename)

def format_float(x, decimals=3):
    return f"{x:.{decimals}f}"

# ------------------------ Load and Preprocess Data ------------------------

dax = yf.download('^GDAXI', start='2014-10-23', end='2019-07-07')
dax = dax[['Close']].dropna().rename(columns={'Close': 'index'})
dax['logreturns'] = np.log(dax['index'] / dax['index'].shift(1))
dax.dropna(inplace=True)
logret = dax['logreturns']
logret.index = dax.index.astype(str)

# ------------------------ KPSS Test ------------------------

stat, pvalue, _, crit = kpss(logret, regression='c', nlags='auto')
print(f"KPSS Statistic: {format_float(stat)}")
print(f"p-value: {format_float(pvalue)}")

# ------------------------ Histogram + KDE + Normal ------------------------

smean = logret.mean()
scal = logret.std(ddof=1)
print(f"Sample Mean: {format_float(smean, 4)}")
print(f"Sample Std Dev: {format_float(scal, 4)}")

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(logret, bins=40, density=True, label='Histogram', alpha=0.6)
kde = sm.nonparametric.KDEUnivariate(logret)
kde.fit()
ax.plot(kde.support, kde.density, label='KDE')
ax.plot(kde.support, norm.pdf(kde.support, loc=smean, scale=scal),
        label=f'Normal PDF (μ={format_float(smean, 4)}, σ={format_float(scal, 4)})')
ax.set_title("Histogram of Log-Returns")
style_legend_outside_bottom(ax, ncol=3)
fig.tight_layout(rect=[0, 0.1, 1, 0.95])
save_figure(fig, "histogram_kde_normal.png")

# ------------------------ ACF/PACF ------------------------

acf_pacf_fig_save(logret, lags=48, both=True, filename="acf_pacf_logret.png")

# ------------------------ Ljung–Box on Returns ------------------------

plot_LB_pvalue(logret, noestimatedcoef=0, nolags=36)
fig = plt.gcf()
fig.patch.set_alpha(0.0)
for ax in fig.axes:
    ax.patch.set_alpha(0.0)
    if ax.get_legend():
        ax.get_legend().get_frame().set_alpha(0.0)
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, framealpha=0.0)
fig.tight_layout(rect=[0, 0.1, 1, 0.95])
save_figure(fig, "lb_pvalue_logret.png")

# ------------------------ ARMA Order Selection ------------------------

choose_arma2(logret, max_p=5, max_q=5, ctrl=1.02)

# ------------------------ Fit ARMA(0,1) ------------------------

arma01 = ARIMA(logret, order=(0, 0, 1), trend='n').fit()
print("\nARMA(0,1) Coefficients:")
for name, val in arma01.params.items():
    ci = arma01.conf_int().loc[name]
    print(f"{name}: {format_float(val)} (95% CI: {format_float(ci[0])}, {format_float(ci[1])})")

# ------------------------ Residual Diagnostics ------------------------

plot_LB_pvalue(arma01.resid, noestimatedcoef=1, nolags=36)
fig = plt.gcf()
fig.patch.set_alpha(0.0)
for ax in fig.axes:
    ax.patch.set_alpha(0.0)
    if ax.get_legend():
        ax.get_legend().get_frame().set_alpha(0.0)
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, framealpha=0.0)
fig.tight_layout(rect=[0, 0.1, 1, 0.95])
save_figure(fig, "lb_pvalue_arma_resid.png")

acf_pacf_fig_save(arma01.resid**2, lags=30, both=True, filename="acf_pacf_arma_resid_squared.png")

# ------------------------ GARCH(1,1) Model ------------------------

garchmod = arch_model(arma01.resid, mean='Zero').fit(disp='off')
print("\nGARCH(1,1) Coefficients:")
for name in garchmod.params.index:
    val = garchmod.params[name]
    ci = garchmod.conf_int().loc[name]
    print(f"{name}: {format_float(val)} (95% CI: {format_float(ci[0])}, {format_float(ci[1])})")

# ------------------------ GARCH Residual Diagnostics ------------------------

garchresid = garchmod.std_resid
acf_pacf_fig_save(garchresid**2, lags=40, both=True, filename="acf_pacf_garchresid_squared.png")

plot_LB_pvalue(garchresid**2, noestimatedcoef=0, nolags=30)
fig = plt.gcf()
fig.patch.set_alpha(0.0)
for ax in fig.axes:
    ax.patch.set_alpha(0.0)
    if ax.get_legend():
        ax.get_legend().get_frame().set_alpha(0.0)
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, framealpha=0.0)
fig.tight_layout(rect=[0, 0.1, 1, 0.95])
save_figure(fig, "lb_pvalue_garch_squared.png")

# QQ Plot with transparent background and legend outside bottom
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
qqplot(garchresid, line='q', fit=True, ax=ax)
ax.set_title("QQ Plot of Standardized GARCH Residuals")
ax.legend(['Data', 'Reference Line'], loc='upper center',
          bbox_to_anchor=(0.5, -0.1), ncol=2, framealpha=0.0)
fig.tight_layout(rect=[0, 0.1, 1, 0.95])
save_figure(fig, "qqplot_garchresid.png")

# ------------------------ Distributional Summary ------------------------

jb_stat, jb_p = jarque_bera(logret)[:2]
print(f"\nJarque–Bera: {format_float(jb_stat)} (p = {format_float(jb_p)})")
print(f"Skewness: {format_float(skew(logret))}")
print(f"Kurtosis: {format_float(kurtosis(logret, fisher=False))}")